# Library

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime
import pyexasol
import psycopg2
import time

In [2]:
import os

In [3]:
start = time.time()

# Folder

In [4]:
ods_fold = 'D:/data/big2/ODS'

# Server

In [5]:
# # server
# # insider db
# wd = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='VSYSD')
# we = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='vsyse')

In [6]:
# exasol db
ws = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYS')
wd = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSD')
we = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='vsyse')

# Load

## 등록정보(CEG_CAR_MIG) 4등급만

In [7]:
# 4m 8.4s
car = wd.export_to_pandas("SELECT VIN, BSPL_STDG_CD, EXHST_GAS_GRD_CD, EXHST_GAS_CERT_NO, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD, VHCL_FBCTN_YMD, VHCL_MNG_NO, VHRNO, EXTGAS_INSP_VLD_YMD, VHCL_OWNR_CL_CD FROM STD_CEG_CAR_MIG WHERE EXHST_GAS_GRD_CD = 'A0504' OR EXHST_GAS_GRD_CD = 'A05T4';")
car_ch_col = {
    'VIN':'차대번호', 
    'BSPL_STDG_CD':'법정동코드', 
    'EXHST_GAS_GRD_CD':'배출가스등급', 
    'EXHST_GAS_CERT_NO':'배출가스인증번호',
    'VHCL_ERSR_YN':'차량말소YN',
    'MANP_MNG_NO':'제원관리번호', 
    'YRIDNW':'차량연식', 
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도', 
    'FRST_REG_YMD':'최초등록일자',
    'VHCL_FBCTN_YMD':'제작일자', 
    'VHCL_MNG_NO':'차량관리번호', 
    'VHRNO':'차량번호',
    'EXTGAS_INSP_VLD_YMD':'검사유효일',
    'VHCL_OWNR_CL_CD':'소유자구분',  
}
carr = car.rename(columns=car_ch_col)
carr.columns

----------------------------------------
Exception occurred during processing of request from ('106.245.232.34', 8563)
Traceback (most recent call last):
  File "c:\ProgramData\Anaconda3\lib\socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "c:\ProgramData\Anaconda3\lib\socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "c:\ProgramData\Anaconda3\lib\socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "c:\ProgramData\Anaconda3\lib\socketserver.py", line 747, in __init__
    self.handle()
  File "C:\Users\kbjung\AppData\Roaming\Python\Python39\site-packages\pyexasol\http_transport.py", line 473, in handle
    self.method_put_compressed()
  File "C:\Users\kbjung\AppData\Roaming\Python\Python39\site-packages\pyexasol\http_transport.py", line 520, in method_put_compressed
    raise e
  File "C:\Users\kbjung\AppData\Roa

ExaQueryError: 
(
    message     =>  ETL-5106: Following error occured while writing data to external connection [https://10.17.1.10:37563/000.gz failed after 734679252 bytes. [OpenSSL SSL_read: Connection reset by peer, errno 104],[56],[Failure when receiving data from the peer]] (Session: 1771277572794810368)
    dsn         =>  dev.openankus.org:8563
    user        =>  sys
    schema      =>  VSYSD
    session_id  =>  1771277572794810368
    code        =>  42636
    query       =>  EXPORT (
SELECT VIN, BSPL_STDG_CD, EXHST_GAS_GRD_CD, EXHST_GAS_CERT_NO, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD, VHCL_FBCTN_YMD, VHCL_MNG_NO, VHRNO, EXTGAS_INSP_VLD_YMD, VHCL_OWNR_CL_CD FROM STD_CEG_CAR_MIG
) INTO CSV
AT 'https://10.17.1.10:37563' FILE '000.gz'
WITH COLUMN NAMES
)


In [ ]:
carr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29998033 entries, 0 to 29998032
Data columns (total 15 columns):
 #   Column    Dtype  
---  ------    -----  
 0   차대번호      object 
 1   법정동코드     int64  
 2   배출가스등급    object 
 3   배출가스인증번호  object 
 4   차량말소YN    object 
 5   제원관리번호    object 
 6   차량연식      int64  
 7   차종        object 
 8   용도        object 
 9   최초등록일자    int64  
 10  제작일자      object 
 11  차량관리번호    object 
 12  차량번호      object 
 13  검사유효일     float64
 14  소유자구분     object 
dtypes: float64(1), int64(3), object(11)
memory usage: 3.4+ GB


## 제원정보(CEG_CAR_SRC_MIG)

In [ ]:
# 4.6s
src = wd.export_to_pandas("SELECT MANP_MNG_NO, FUEL_CD, VHCTY_TY_CD2, MNFCTR_NM, VHCNM, VHCL_FRM, EGIN_TY, VHCTY_CL_CD, TOTL_WGHT, CRYNG_WGHT, DSPLVL, EGIN_OTPT FROM STD_CEG_CAR_SRC_MIG;")
src_ch_col = {
    'MANP_MNG_NO':'제원관리번호', 
    'FUEL_CD':'연료',
    'VHCTY_TY_CD2':'차종유형', 
    'MNFCTR_NM':'제작사명', 
    'VHCNM':'차명', 
    'VHCL_FRM':'자동차형식', 
    'EGIN_TY':'엔진형식', 
    'VHCTY_CL_CD':'차종분류',
    'TOTL_WGHT':'총중량',
    'CRYNG_WGHT':'적재중량',
    'DSPLVL':'배기량', 
    'EGIN_OTPT':'엔진출력',
}
srcr = src.rename(columns=src_ch_col)
srcr.columns

Index(['제원관리번호', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식', '차종분류', '총중량',
       '적재중량', '배기량', '엔진출력'],
      dtype='object')

In [ ]:
srcr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561153 entries, 0 to 561152
Data columns (total 12 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   제원관리번호  561153 non-null  object 
 1   연료      518235 non-null  object 
 2   차종유형    546198 non-null  object 
 3   제작사명    544020 non-null  object 
 4   차명      561051 non-null  object 
 5   자동차형식   531308 non-null  object 
 6   엔진형식    508601 non-null  object 
 7   차종분류    546233 non-null  object 
 8   총중량     557481 non-null  float64
 9   적재중량    458331 non-null  float64
 10  배기량     525597 non-null  float64
 11  엔진출력    486280 non-null  float64
dtypes: float64(4), object(8)
memory usage: 51.4+ MB


## 정기검사(STD_TB_JGT_HIS)

In [ ]:
# 3m 29.1s
jgt = wd.export_to_pandas("SELECT VIN, FDRM_INSP_INSP_MTHD_CD, FDRM_INSP_KND_CD, FDRM_INSP_JGMT, FDRM_NLOD_SMO_MSTVL1, FDRM_NLOD_SMO_MSTVL2, FDRM_NLOD_SMO_MSTVL3, FDRM_NLOD_SMO_JT_YN1, FDRM_INSP_YMD, FDRM_DRVNG_DSTNC, FDRM_NLOD_SMO_PRMVL1 FROM STD_TB_JGT_HIS;")
jgt_ch_col = {
    'VIN':'차대번호', 
    'FDRM_INSP_INSP_MTHD_CD':'검사방법', 
    'FDRM_INSP_KND_CD':'검사종류', 
    'FDRM_INSP_JGMT':'검사판정', 
    'FDRM_NLOD_SMO_MSTVL1':'무부하매연측정치1', 
    'FDRM_NLOD_SMO_MSTVL2':'무부하매연측정치2', 
    'FDRM_NLOD_SMO_MSTVL3':'무부하매연측정치3', 
    'FDRM_NLOD_SMO_JT_YN1':'무부하매연판정1', 
    'FDRM_INSP_YMD':'검사일자',
    'FDRM_DRVNG_DSTNC':'주행거리',
    'FDRM_NLOD_SMO_PRMVL1':'무부하매연허용치1', 
}
jgtr = jgt.rename(columns=jgt_ch_col)
jgtr.columns

Index(['차대번호', '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1'],
      dtype='object')

In [ ]:
jgtr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64316863 entries, 0 to 64316862
Data columns (total 11 columns):
 #   Column     Dtype  
---  ------     -----  
 0   차대번호       object 
 1   검사방법       object 
 2   검사종류       object 
 3   검사판정       object 
 4   무부하매연측정치1  float64
 5   무부하매연측정치2  float64
 6   무부하매연측정치3  float64
 7   무부하매연판정1   object 
 8   검사일자       int64  
 9   주행거리       float64
 10  무부하매연허용치1  float64
dtypes: float64(5), int64(1), object(5)
memory usage: 5.3+ GB


## 정밀검사(STD_TB_EET_HIS_ME)

In [ ]:
# 7m 3.2s
eet = wd.export_to_pandas("SELECT VIN, PRCINSP_MSRMT_MTHD_CD, PRCINSP_KND_CD, PRCINSP_JGMT, PREC_NLOD_SMO_MSTVL1, PREC_NLOD_SMO_MSTVL2, PREC_NLOD_SMO_MSTVL3, PREC_NLOD_SMO_JT_YN1, PRCINSP_YMD, PRCINSP_DRVNG_DSTNC, PREC_NLOD_SMO_PRMVL1 FROM STD_TB_EET_HIS_ME;")
eet_ch_col = {
    'VIN':'차대번호', 
    'PRCINSP_MSRMT_MTHD_CD':'검사방법', 
    'PRCINSP_KND_CD':'검사종류', 
    'PRCINSP_JGMT':'검사판정', 
    'PREC_NLOD_SMO_MSTVL1':'무부하매연측정치1', 
    'PREC_NLOD_SMO_MSTVL2':'무부하매연측정치2', 
    'PREC_NLOD_SMO_MSTVL3':'무부하매연측정치3', 
    'PREC_NLOD_SMO_JT_YN1':'무부하매연판정1', 
    'PRCINSP_YMD':'검사일자',
    'PRCINSP_DRVNG_DSTNC':'주행거리', 
    'PREC_NLOD_SMO_PRMVL1':'무부하매연허용치1', 
}
eetr = eet.rename(columns=eet_ch_col)
eetr.columns

Index(['차대번호', '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1'],
      dtype='object')

In [ ]:
eetr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99608917 entries, 0 to 99608916
Data columns (total 11 columns):
 #   Column     Dtype  
---  ------     -----  
 0   차대번호       object 
 1   검사방법       object 
 2   검사종류       object 
 3   검사판정       object 
 4   무부하매연측정치1  float64
 5   무부하매연측정치2  float64
 6   무부하매연측정치3  float64
 7   무부하매연판정1   object 
 8   검사일자       int64  
 9   주행거리       float64
 10  무부하매연허용치1  float64
dtypes: float64(5), int64(1), object(5)
memory usage: 8.2+ GB


## 법정동코드(STD_BJCD_INFO)

In [ ]:
# 1.3s
code = wd.export_to_pandas("SELECT STDG_CD, STDG_CTPV_NM, STDG_SGG_NM, STDG_CTPV_CD, STDG_SSG_CD FROM STD_BJCD_INFO;")
code_ch_col = {
    'STDG_CD':'법정동코드', 
    'STDG_CTPV_NM':'시도', 
    'STDG_SGG_NM':'시군구', 
    'STDG_CTPV_CD':'시도코드',
    'STDG_SSG_CD':'시군구코드', # SSG -> SGG
}
coder = code.rename(columns=code_ch_col)
coder.columns

Index(['법정동코드', '시도', '시군구', '시도코드', '시군구코드'], dtype='object')

In [ ]:
coder.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47916 entries, 0 to 47915
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   법정동코드   47916 non-null  int64 
 1   시도      47916 non-null  object
 2   시군구     47892 non-null  object
 3   시도코드    47916 non-null  int64 
 4   시군구코드   47916 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 1.8+ MB


## 노후차 조기폐차 관리정보(수도권)(STD_DLM_TB_ERP_EARLY_ERASE_AEA)

In [ ]:
# 2.8s
aea = wd.export_to_pandas("SELECT VIN, ELPDSRC_STTS_CD, ELPDSRC_LST_APRV_YN, ERSR_YMD FROM STD_DLM_TB_ERP_EARLY_ERASE_AEA;")
aea_ch_col = {
    'VIN':'차대번호', 
    'ELPDSRC_STTS_CD':'조기폐차상태코드', 
    'ELPDSRC_LST_APRV_YN':'조기폐차최종승인YN', 
    'ERSR_YMD':'말소일자', 
}
aear = aea.rename(columns=aea_ch_col)
aear.columns

Index(['차대번호', '조기폐차상태코드', '조기폐차최종승인YN', '말소일자'], dtype='object')

In [ ]:
aear.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804082 entries, 0 to 804081
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   차대번호        804082 non-null  object 
 1   조기폐차상태코드    804082 non-null  object 
 2   조기폐차최종승인YN  683874 non-null  object 
 3   말소일자        689973 non-null  float64
dtypes: float64(1), object(3)
memory usage: 24.5+ MB


## 노후차 조기폐차 관리정보(수도권외)(STD_DLM_TB_ERP_EARLY_ERASE_LGV)

In [ ]:
# 1.9s
lgv = wd.export_to_pandas("SELECT VIN, ELPDSRC_STTS_CD, ELPDSRC_LST_APRV_YN, ERSR_YMD FROM STD_DLM_TB_ERP_EARLY_ERASE_LGV;")
lgv_ch_col = {
    'VIN':'차대번호', 
    'ELPDSRC_STTS_CD':'조기폐차상태코드', 
    'ELPDSRC_LST_APRV_YN':'조기폐차최종승인YN', 
    'ERSR_YMD':'말소일자',  
}
lgvr = lgv.rename(columns=lgv_ch_col)
lgvr.columns

Index(['차대번호', '조기폐차상태코드', '조기폐차최종승인YN', '말소일자'], dtype='object')

In [ ]:
lgvr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373371 entries, 0 to 373370
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   차대번호        373371 non-null  object 
 1   조기폐차상태코드    373371 non-null  object 
 2   조기폐차최종승인YN  160690 non-null  object 
 3   말소일자        175443 non-null  float64
dtypes: float64(1), object(3)
memory usage: 11.4+ MB


## 저감장치 부착이력(STD_DLM_TB_ERP_ATT_HIS)

In [ ]:
# 2.7s
att = wd.export_to_pandas("SELECT VIN, RDCDVC_SE_CD FROM STD_DLM_TB_ERP_ATT_HIS;")
att_ch_col = {
    'VIN':'차대번호', 
    'RDCDVC_SE_CD':'저감장치구분', 
}
attr = att.rename(columns=att_ch_col)
attr.columns

Index(['차대번호', '저감장치구분'], dtype='object')

In [ ]:
attr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010946 entries, 0 to 1010945
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   차대번호    1010946 non-null  object
 1   저감장치구분  1010946 non-null  object
dtypes: object(2)
memory usage: 15.4+ MB


## 등록이력(CEG_CAR_HISTORY_MIG)

In [ ]:
# 1m 13.2s
his = ws.export_to_pandas("SELECT VHCL_ERSR_YN, CHNG_DE, VHMNO FROM CEG_CAR_HISTORY_MIG;")
his_ch_col = {
    'VHCL_ERSR_YN':'차량말소YN', 
    'CHNG_DE':'변경일자',
    'VHMNO':'차량관리번호'
}
hisr = his.rename(columns=his_ch_col)
hisr.columns

Index(['차량말소YN', '변경일자', '차량관리번호'], dtype='object')

In [ ]:
# edb_id = 'vsysd'
# edb_database = 'edb'
# edb_port = 5444
# edb_url = '172.29.135.50'
# edb_pwd = 'vsyswynn'
# conn = psycopg2.connect(dbname=edb_database, user=edb_id, password=edb_pwd, host=edb_url, port=edb_port)
# cur = conn.cursor()
# sql = 'select VHCL_ERSR_YN, CHNG_DE, VHMNO from vsysd.ods_ceg_car_history_mig'
# cur.execute(sql)
# his = pd.DataFrame(cur.fetchall())
# his.columns = [desc[0].upper() for desc in his.description]
# his_ch_col = {
#     'VHCL_ERSR_YN':'차량말소YN', 
#     'CHNG_DE':'변경일자',
#     'VHMNO':'차량관리번호'
# }
# hisr = his.rename(columns=his_ch_col)
# hisr.columns

In [ ]:
hisr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50044247 entries, 0 to 50044246
Data columns (total 3 columns):
 #   Column  Dtype  
---  ------  -----  
 0   차량말소YN  object 
 1   변경일자    float64
 2   차량관리번호  object 
dtypes: float64(1), object(2)
memory usage: 1.1+ GB


## 비상시 및 계절제 단속발령(N_IS_ISSUE_DISCLOSURE)

In [ ]:
# 1m 27.5s
isdis = pd.read_csv(os.path.join(ods_fold, 'ODS_N_IS_ISSUE_DISCLOSURE(col_sliced).csv'), low_memory=False)
isdis_ch_col = {
    'REGLT_NO':'적발번호', 
    'GNFD_NO':'발령번호', 
    'REG_SIDO_CD':'등록시도코드', 
    'REG_SIGNGU_CD':'등록시군구코드', 
    'VIN':'차대번호',
    'REGLT_AREA_CD':'적발지역코드', 
}
isdisr = isdis.rename(columns=isdis_ch_col)
isdisr.columns

Index(['적발번호', '발령번호', '등록시도코드', '등록시군구코드', '적발지역코드', '차대번호'], dtype='object')

In [ ]:
# sql = "select REGLT_NO, GNFD_NO, VIN, REG_SIDO_CD, REG_SIGNGU_CD, REGLT_AREA_CD from vsysd.n_is_issue_disclosure"
# cur.execute(sql)
# isdis = pd.DataFrame(cur.fetchall())

# isdis.columns = [desc[0].upper() for desc in isdis.description]
# isdis_ch_col = {
#     'REGLT_NO':'적발번호', 
#     'GNFD_NO':'발령번호', 
#     'REG_SIDO_CD':'등록시도코드', 
#     'REG_SIGNGU_CD':'등록시군구코드', 
#     'VIN':'차대번호',
#     'REGLT_AREA_CD':'적발지역코드', 
# }
# isdisr = isdis.rename(columns=isdis_ch_col)
# isdisr.columns

In [ ]:
isdisr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81512621 entries, 0 to 81512620
Data columns (total 6 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   적발번호     int64 
 1   발령번호     int64 
 2   등록시도코드   int64 
 3   등록시군구코드  int64 
 4   적발지역코드   int64 
 5   차대번호     object
dtypes: int64(5), object(1)
memory usage: 3.6+ GB


## 운행제한 발령정보(N_IS_ISSUE)

In [ ]:
# 1s
isis = pd.read_csv(os.path.join(ods_fold, 'ODS_N_IS_ISSUE.csv'), low_memory=False, usecols=['GNFD_NO', 'DNSTY_STDR_ID', 'TY_STDR_ID'])
isis_ch_col = {
    'GNFD_NO':'발령번호', 
    'DNSTY_STDR_ID':'농도기준아이디', 
    'TY_STDR_ID':'유형기준아이디', 
}
isisr = isis.rename(columns=isis_ch_col)
isisr.columns

Index(['발령번호', '농도기준아이디', '유형기준아이디'], dtype='object')

In [ ]:
# sql = "select GNFD_NO, TY_STDR_ID, DNSTY_STDR_ID from vsysd.n_is_issue"
# cur.execute(sql)
# isis = pd.DataFrame(cur.fetchall())

# isis.columns = [desc[0].upper() for desc in isis.description]
# isis_ch_col = {
#     'GNFD_NO':'발령번호', 
#     'DNSTY_STDR_ID':'농도기준아이디', 
#     'TY_STDR_ID':'유형기준아이디', 
# }
# isisr = isis.rename(columns=isis_ch_col)
# isisr.columns

In [ ]:
isisr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1785 entries, 0 to 1784
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   발령번호     1785 non-null   int64 
 1   농도기준아이디  1785 non-null   object
 2   유형기준아이디  1785 non-null   object
dtypes: int64(1), object(2)
memory usage: 42.0+ KB


## N_IS_PENALTY

In [ ]:
# 13.4s
ispe = pd.read_csv(os.path.join(ods_fold, 'N_IS_PENALTY.csv'), low_memory=False, sep='|', usecols=['REGLT_NO', 'REGLT_DE'])
ispe_ch_col = {
    'REGLT_NO':'적발번호', 
    'REGLT_DE':'단속일', 
}
isper = ispe.rename(columns=ispe_ch_col)
isper.columns

Index(['단속일', '적발번호'], dtype='object')

In [ ]:
# sql = "select REGLT_NO, REGLT_DE from vsysd.n_is_penalty"
# cur.execute(sql)
# ispe = pd.DataFrame(cur.fetchall())
# ispe.columns = [desc[0].upper() for desc in ispe.description]
# ispe_ch_col = {
#     'REGLT_NO':'적발번호', 
#     'REGLT_DE':'단속일', 
# }
# isper = ispe.rename(columns=ispe_ch_col)
# isper.columns

In [ ]:
isper.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10790105 entries, 0 to 10790104
Data columns (total 2 columns):
 #   Column  Dtype
---  ------  -----
 0   단속일     int64
 1   적발번호    int64
dtypes: int64(2)
memory usage: 164.6 MB


## 운행제한 단속정보(N_US_DISCLOSURE)

In [ ]:
# 14.2s
usdis = pd.read_csv(os.path.join(ods_fold, 'ODS_N_US_DISCLOSURE.csv'), low_memory=False, usecols=['NO', 'VIN', 'REG_SIDO_CD', 'REG_SIGNGU_CD', 'DISCL_TY', 'REGLT_AREA_CD'])
usdis_ch_dict = {
    'NO':'번호', 
    'VIN':'차대번호', 
    'REG_SIDO_CD':'등록시도코드', 
    'REG_SIGNGU_CD':'등록시군구코드', 
    'DISCL_TY':'적발유형', 
    'REGLT_AREA_CD':'단속지역코드', 
}
usdisr = usdis.rename(columns=usdis_ch_dict)
usdisr.columns

Index(['번호', '등록시도코드', '등록시군구코드', '적발유형', '단속지역코드', '차대번호'], dtype='object')

In [ ]:
# sql = 'select "NO", VIN, DISCL_TY, REGLT_AREA_CD, REG_SIDO_CD, REG_SIGNGU_CD from vsysd.n_us_disclosure'
# cur.execute(sql)
# usdis = pd.DataFrame(cur.fetchall())
# usdis.columns = [desc[0].upper() for desc in usdis.description]
# usdis_ch_dict = {
#     'NO':'번호', 
#     'VIN':'차대번호', 
#     'REG_SIDO_CD':'등록시도코드', 
#     'REG_SIGNGU_CD':'등록시군구코드', 
#     'DISCL_TY':'적발유형', 
#     'REGLT_AREA_CD':'단속지역코드', 
# }
# usdisr = usdis.rename(columns=usdis_ch_dict)
# usdisr.columns

In [ ]:
usdisr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3090035 entries, 0 to 3090034
Data columns (total 6 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   번호       int64 
 1   등록시도코드   int64 
 2   등록시군구코드  int64 
 3   적발유형     object
 4   단속지역코드   int64 
 5   차대번호     object
dtypes: int64(4), object(2)
memory usage: 141.5+ MB


## N_US_PENALTY

In [ ]:
# 0.5s
uspe = pd.read_csv(os.path.join(ods_fold, 'N_US_PENALTY.csv'), low_memory=False, sep='|', usecols=['NO', 'REGLT_CNT', 'REGLT_YM'])
uspe_ch_dict = {
    'NO':'번호', 
    'REGLT_CNT':'적발건수', 
    'REGLT_YM':'적발년월'
}
usper = uspe.rename(columns=uspe_ch_dict)
usper.columns

Index(['적발년월', '적발건수', '번호'], dtype='object')

In [ ]:
# sql = 'select "NO", REGLT_CNT, REGLT_YM FROM from vsysd.n_us_penalty'
# cur.execute(sql)
# uspe = pd.DataFrame(cur.fetchall())
# cur.close()
# conn.close()
# uspe.columns = [desc[0].upper() for desc in uspe.description]
# uspe_ch_dict = {
#     'NO':'번호', 
#     'REGLT_CNT':'적발건수', 
#     'REGLT_YM':'적발년월'
# }
# usper = uspe.rename(columns=uspe_ch_dict)
# usper.columns

In [ ]:
usper.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 443309 entries, 0 to 443308
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   적발년월    443309 non-null  int64
 1   적발건수    443309 non-null  int64
 2   번호      443309 non-null  int64
dtypes: int64(3)
memory usage: 10.1 MB


## RH에서 제공한 법정동코드

In [ ]:
# 1.1s
rh = we.export_to_pandas("SELECT DONG_CODE, CTPRVN_NM, SIGNGU_NM FROM TB_MAPDATA;")
rh = rh.rename(columns={
    'DONG_CODE':'법정동코드_rh', 
    'CTPRVN_NM':'시도', 
    'SIGNGU_NM':'시군구'
    })
rh.columns

Index(['법정동코드_rh', '시도', '시군구'], dtype='object')

In [ ]:
rh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   법정동코드_rh  278 non-null    int64 
 1   시도        278 non-null    object
 2   시군구       260 non-null    object
dtypes: int64(1), object(2)
memory usage: 6.6+ KB


## 4등급 result(for DPF유무)

In [ ]:
# 3.0s
rs = we.export_to_pandas("SELECT 차대번호, DPF유무_수정 FROM GRD4_RESULT;")
rs.columns

Index(['차대번호', 'DPF유무_수정'], dtype='object')

In [ ]:
rs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1502235 entries, 0 to 1502234
Data columns (total 2 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   차대번호      1502235 non-null  object
 1   DPF유무_수정  1153813 non-null  object
dtypes: object(2)
memory usage: 22.9+ MB


## STD_KOSIS

In [ ]:
# 1.2s
kosis = we.export_to_pandas("SELECT CTPV, SGG, VHCTY_CD, DY_AVRG_DRVNG_DSTNC FROM STD_KOSIS;")
kosis_ch_col = {
    'CTPV':'시도', 
    'SGG':'시군구', 
    'VHCTY_CD':'차종', 
    'DY_AVRG_DRVNG_DSTNC':'일일평균주행거리', 
}
kosisr = kosis.rename(columns=kosis_ch_col)
kosisr.columns

Index(['시도', '시군구', '차종', '일일평균주행거리'], dtype='object')

In [ ]:
kosisr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1230 entries, 0 to 1229
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   시도        1230 non-null   object 
 1   시군구       1230 non-null   object 
 2   차종        1230 non-null   object 
 3   일일평균주행거리  1230 non-null   float64
dtypes: float64(1), object(3)
memory usage: 38.6+ KB


# Preprocessing

## 중복 차대번호 제거

In [ ]:
carr.shape

(29998033, 15)

In [ ]:
# 
carr['최초등록일자'] = pd.to_numeric(carr['최초등록일자'], errors='coerce')
carr = carr.sort_values('최초등록일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
carr.shape

(29984639, 15)

## 등록정보(CEG_CAR_MIG)

### 배출가스등급 코드 변환

In [ ]:
grd_dict = {
    'A0501':'1', 
    'A0502':'2', 
    'A0503':'3', 
    'A0504':'4', 
    'A0505':'5', 
    'A05T2':'2',
    'A05T3':'3', 
    'A05T4':'4', 
    'A05T5':'5', 
    'A05X':'X', 
}
carr['배출가스등급'] = carr['배출가스등급'].replace(grd_dict)
carr['배출가스등급'].unique()

array(['X', '2', '1', '3', '5', '4', nan], dtype=object)

### 차종 코드 변환

In [ ]:
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
carr['차종'] = carr['차종'].replace(cd_dict)
carr['차종'].unique()

array(['승용', '화물', '승합', '특수', nan], dtype=object)

### 용도 코드 변환

In [ ]:
purps_dict = {
    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용',
}
carr['용도'] = carr['용도'].replace(purps_dict)
carr['용도'].unique()

array(['개인용', '영업용', '관용'], dtype=object)

### 소유자구분 코드 변환

In [ ]:
ownr_dict = {
    'A27B':'사업자', 
    'A27F':'외국인', 
    'A27L':'법인', 
    'A27O':'기타', 
    'A27R':'주민', 
}
carr['소유자구분'] = carr['소유자구분'].replace(ownr_dict)
carr['소유자구분'].unique()

array(['법인', '주민', '사업자', '기타', '외국인'], dtype=object)

### 등록정보 말소 제거

In [ ]:
carr.shape

In [ ]:
carm = carr[carr['차량말소YN'] == 'N'].reset_index(drop=True)
carm.shape

## 제원정보(CEG_CAR_SRC_MIG)

### 연료 코드 변환

In [ ]:
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG(액화석유가스)', 
    'A90GH':'휘발유 하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유 하이브리드', 
    'A92CN':'CNG(압축천연가스)', 
    'A93HD':'수소', 
    'A92LH':'LPG 하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG 하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG(액화천연가스)', 
    'A90PH':'플러그인 하이브리드', 
}
srcr['연료'] = srcr['연료'].replace(fuel_dict)
srcr['연료'].unique()

array(['경유', '휘발유', 'CNG(압축천연가스)', nan, 'LPG(액화석유가스)', '기타연료',
       '휘발유 하이브리드', '전기', '알코올', '태양열', 'CNG 하이브리드', '등유', '경유 하이브리드',
       'LPG 하이브리드', '수소', 'LNG(액화천연가스)'], dtype=object)

### 차종유형 코드 변환

In [ ]:
ty_dict = {
    'A30C':'경형', 
    'A30L':'대형', 
    'A30M':'중형', 
    'A30S':'소형',
}
srcr['차종유형'] = srcr['차종유형'].replace(ty_dict)
srcr['차종유형'].unique()

array(['대형', '소형', '중형', '경형', nan], dtype=object)

## 등록&제원 병합

In [ ]:
carm.shape

In [ ]:
cs = carm.merge(srcr, on='제원관리번호', how='left')
cs.shape

### 정기&정밀(STD_TB_JGT_HIS, STD_TB_EET_HIS_ME)

#### 데이터셋 병합

In [ ]:
jgtr.shape, eetr.shape

((64316863, 11), (99608917, 11))

In [ ]:
jgtr.shape[0] + eetr.shape[0]

163925780

In [ ]:
# 
ins = pd.concat([jgtr, eetr], ignore_index=True)
ins.shape

(163925780, 11)

#### 최근 검사만 추출

In [ ]:
insm = ins.sort_values('검사일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
insm.shape

(34114272, 11)

#### 검사종류 코드 변환

In [ ]:
knd_dict = {
    'A21P01':'정밀검사', 
    'A1904':'정기검사', 
    'A21T06':'종합검사(경과)', 
    'A21T05':'종합검사', 
    'A21S01':'특정검사', 
    'A1902':'재검사(2부재검사)',
    'A21R02':'2부재검사(종합)', 
    'A21P02':'정밀검사(경과)', 
    'A1903':'정기(경과)검사', 
    'A21S04':'특정재검사(배출)', 
    'A21E01':'배출재검사', 
    'A21R01':'1부재검사(종합)',
    'A21C02':'구조변경검사', 
    'A21002':'관능재검사(임시)', 
    'A21S03':'특정재검사(관능)', 
    'A21S02':'특정검사(경과)', 
    'A1901':'재검사(1부재검사)', 
    'A21T02':'종합(정밀)',
    'A21E02':'배출재검사(구변)', 
    'A21T01':'종합(경과:정밀)', 
    'A21T04':'종합(정밀) 2부재검사', 
    'A21001':'관능재검사', 
    'A21T03':'종합(정밀) 1부재검사', 
    'A21TMP':'임시검사', 
}
insm['검사종류'] = insm['검사종류'].replace(knd_dict)
insm['검사종류'].unique()

array(['종합검사', '종합검사(경과)', '1부재검사(종합)', '2부재검사(종합)', '정기검사', '재검사(1부재검사)',
       '재검사(2부재검사)', '정기(경과)검사', '배출재검사(구변)', nan, '구조변경검사',
       '종합(정밀) 1부재검사', '종합(정밀) 2부재검사', '종합(정밀)', '종합(경과:정밀)', '정밀검사',
       'A21C01', '특정검사', '특정재검사(배출)', '임시검사', '배출재검사', '관능재검사',
       '특정재검사(관능)', '정밀검사(경과)', '특정검사(경과)', 'A21R05', 'A21R06',
       '관능재검사(임시)', 'A21E03'], dtype=object)

#### 검사방법 코드 변환

In [ ]:
mth_dict = {
    'A18A':'무부하검사(TSI)', 
    'A18B':'무부하검사(급가속)', 
    'A18C':'무부하검사(정지가동)',
    'A2301':'무부하검사(급가속)', 
    'A2302':'무부하검사(정지가동)',
    'A2303':'무부하검사(TSI)', 
    'A2304':'부하검사(LUG DOWN)', 
    'A2305':'부하검사(ASM-Idling)', 
    'A2306':'부하검사(KD-147)', 
    'A2307':'부하검사(ASM2525)', 
}
insm['검사방법'] = insm['검사방법'].replace(mth_dict)
insm['검사방법'].unique()

array(['무부하검사(급가속)', '부하검사(KD-147)', '부하검사(ASM-Idling)', '부하검사(LUG DOWN)',
       '무부하검사(TSI)', '무부하검사(정지가동)', '부하검사(ASM2525)'], dtype=object)

### 저감장치 부착이력(STD_DLM_TB_ERP_ATT_HIS)

#### 저감장치구분 코드 변환

In [ ]:
rdcdvc_dict = {
    'A1001':'1종', 
    'A1002':'2종', 
    'A1003':'3종', 
    'A1004':'1종+SCR', 
    'A1005':'엔진개조', 
    'A1006':'엔진교체',
    'A1007':'삼원촉매',
}
attr['저감장치구분'] = attr['저감장치구분'].replace(rdcdvc_dict)
attr['저감장치구분'].unique()

array(['1종', '3종', '엔진개조', '엔진교체', '2종', '1종+SCR', '삼원촉매'], dtype=object)

#### 저감장치 부착유무 추가

In [ ]:
attr.loc[(attr['저감장치구분'] == '1종') | (attr['저감장치구분'] == '1종+SCR'), 'DPF_YN'] = '유'
attr['DPF_YN'].unique()

array(['유', nan], dtype=object)

## 등록&제원 병합

In [ ]:
carr.shape

(29984639, 15)

In [ ]:
# 14.3s
total = carr.merge(srcr, on='제원관리번호', how='left')
total.shape

MemoryError: Unable to allocate 1.12 GiB for an array with shape (5, 29984639) and data type object

## 등록&제원&정기&정밀

In [87]:
# 1m 56.3s
total = total.merge(insm, on='차대번호', how='left')
total.shape

(29984639, 36)

## 등록&제원&정기&정밀&저감

In [88]:
attr.columns

Index(['차대번호', '저감장치구분', 'DPF_YN'], dtype='object')

In [89]:
total = total.merge(attr[['차대번호', 'DPF_YN']], on='차대번호', how='left')
total.shape

MemoryError: Unable to allocate 1.12 GiB for an array with shape (5, 29985076) and data type float64

## 등록&제원&정기&정밀&저감&법정동코드

In [ ]:
total = total.merge(coder, on='법정동코드', how='left')
total.shape

## 등록정보 말소 제거

In [83]:
total.shape

(29984639, 15)

In [84]:
carm = total[total['차량말소YN'] == 'N'].reset_index(drop=True)
carm.shape

(26203793, 15)

# Analysis

In [ ]:
print(start - time.time())

# Code end